# Importing tools

In [1]:
# pip install ortools
# pip install networkx
import networkx as nx
import time

# Importing libraries

In [2]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np

# Plotting the graph

In [3]:
def plot_graph(graph):
    g = nx.Graph()
    A = np.array(graph)
    g = nx.from_numpy_matrix(A, create_using=nx.Graph)

    pos = nx.spring_layout(g)
    labels = nx.get_edge_attributes(g,'weight')
    nx.draw(g, pos, edge_labels=labels, with_labels=True)

# Creating data models

In [4]:
# creating the data model for distance matrix
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = [
        [
            0, 9, 11, 4, 8, 10, 6
        ],
        [
            9, 0, 18, 5, 10, 13, 11
        ],
        [
            11, 18, 0, 12, 10, 11, 7
        ],
        [
            4, 5, 12, 0, 5, 8, 7
        ],
        [
            8, 10, 10, 5, 0, 4, 3
        ],
        [
            10, 13, 11, 8, 4, 0, 4
        ],
        [
            6, 11, 7, 7, 3, 4, 0
        ],
    ]
    data['demands'] = [0, 8, 25, 17, 13, 16, 21]
    data['vehicle_capacities'] = [25, 25]
    data['num_vehicles'] = 2
    data['depot'] = 0
    return data

# Printing the solution

In [5]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    # Display dropped nodes.
    dropped_nodes = 'Dropped nodes:'
    for node in range(routing.Size()):
        if routing.IsStart(node) or routing.IsEnd(node):
            continue
        if solution.Value(routing.NextVar(node)) == node:
            dropped_nodes += ' {}'.format(manager.IndexToNode(node))
    print(dropped_nodes)
    
    total_distance = 0
    total_load = 0
    max_route_distance = 0
    
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))
    max_route_distance = max(route_distance, max_route_distance)
#     print('Maximum of the route distances: {}m'.format(max_route_distance))

# Main function

In [6]:
def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        150,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)
    
    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')
    # Allow to drop nodes.
    penalty = 3000
    for node in range(1, len(data['distance_matrix'])):
        routing.AddDisjunction([manager.NodeToIndex(node)], penalty)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print('No solution found !')


if __name__ == '__main__':
    start_time = time.time()
    main()
    print("---------------")
    print("Total time taken :: %s seconds" % (time.time() - start_time))

Objective: 10830
Dropped nodes: 2 4 5
Route for vehicle 0:
 0 Load(0) ->  6 Load(21) ->  0 Load(21)
Distance of the route: 12m
Load of the route: 21

Route for vehicle 1:
 0 Load(0) ->  3 Load(17) ->  1 Load(25) ->  0 Load(25)
Distance of the route: 18m
Load of the route: 25

Total distance of all routes: 30m
Total load of all routes: 46
---------------
Total time taken :: 1.1053683757781982 seconds
